# 中文文字分類

## ```Da-Wei Chiang```

## ```文字採擷(Text Mining)```

- 意義 : 從非結構化資料中找出使用者感興趣或有用模式的過程
- 定義 : 從大量文字資料中取出事前未知的、可了解的、最後可用的知識的過程
- 文字採擷分為以下幾個領域 : 
    - 搜索和資訊檢索(IR):儲存和文字文件的檢索，如:搜尋引擎、關鍵字搜尋
    - 文字分群 : 使用分群，對詞彙、片段、段落進行歸類
    - 文字分類 : 對詞彙、片段、段落進行歸類，在資料採擷分類方法上進行建模
    - Web採擷 : 網際網路上進行資料和文字採擷
    - 資訊取出(IE) : 從非結構化資料中識別與分析有關的事實和關係
    - 自然語言處理(NLP) : 從語言、語法的角度發現語言最本質的結構所產生的意義
    - 概念分析 : 將單字和子句按語意分成幾個相似的組

## ```文字分類```

- 文字分類的方法分為以下兩種
    - 模式系統(專家評估)
    - 分類模型(機器學習)
- 文字分類的應用
    - 文字檢索、垃圾郵件過濾、網頁分層目錄...等等

## ```中文文字分類步驟```

- 中文的文字分類流程，主要包含以下幾個步驟
    - 前置處理 : 去除文字雜訊，如文字格式轉換、去除HTML標籤..等等
    - 分詞 : 為中文分詞，並去除停用詞 
    - 建置詞的向量空間 : 統計文字詞頻，產生文字的向量空間
    - 加權策略 : 使用TF-IDF發現特徵詞，找出文件主要特徵
    - 分類器 : 使用演算法訓練分類器
    - 評價分類結果 : 分類器的測試結果分析
    

## ```中文文字分類步驟一 : 前置處理```

- 文字的前置處理又分為以下幾個步驟
    - 選擇處理文字的範圍
    - 建立文字語料庫
        - 訓練集語料
        - 測試集語料
    - 文字格式轉換
    - 檢測邊界:標示句子的結束

## ```中文文字分類步驟二 : 分詞```

- 中文分詞一般使用的演算法為
    - ```條件隨機場(CRF):這```是一種機率模型
- 商用的分詞工具
    - [北京理工大學的中文分詞系統](http://ictclas.nlpir.org/)
    - [哈爾濱大學的中文分詞系統](http://www.ltp-cloud.com/intro/)
    - [Ansj的中文分詞系統](http://www.nlpcn.org/)
- ```python中使用的分詞套件為jieba```

## ```中文文字分類步驟二 : 分詞(程式範例)```

- 對文件中每個文字檔進行分詞程式如下

In [5]:
from lxml import etree,html 
import jieba 
import os

corpus_path = "train_corpus_small/"   #原始檔案路徑
seg_path = "train_corpus_seg/"        #處理後的檔案路徑
catelist = os.listdir(corpus_path)

def readfile(path):
    fp = open(path,"rb")
    content = fp.read()
    fp.close()
    return content

def savefile(savepath,content):
    fp = open(savepath,"wb")
    fp.write(content)
    fp.close()

for mydir in catelist:
    class_path = corpus_path+mydir+"/"
    seg_dir = seg_path+mydir+"/"
    if not os.path.exists(seg_dir):
        os.makedirs(seg_dir)
    file_list = os.listdir(class_path)
    for file_path in file_list:
        fullname = class_path+file_path
        content = readfile(fullname).strip()
        content = content.decode().replace("\r\n","").strip()
        content_seg = jieba.cut(content)
        savefile(seg_dir+file_path," ".join(content_seg).encode())
print("分詞結束")         

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\MIRDEX~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.339 seconds.
Prefix dict has been built succesfully.


分詞結束


## ```中文文字分類步驟二 : 分詞```

- 分詞後的文字資訊為了後續建立向量空間模型的方便，需將資訊轉為文字向量資訊並物件化(資料儲存的格式化)
- 轉換的過程使用```scikit-learn中的bunch```資料結構

## ```中文文字分類步驟二 : 分詞(程式範例)```

- 使用```sickit-learn的bunch```資料結構將文字資訊物件化程式如下

In [6]:
from sklearn.datasets.base import Bunch
import pickle

bunch = Bunch(target_name=[],label=[],filenames=[],contents=[]) #宣告物件格式

wordbag_path = "train_word_bag/train_set.dat"    #格式化之後儲存的路徑，以pickle的方式儲存
seg_path = "train_corpus_seg/"       #分詞後，要進行格式化的檔案路徑

catelist = os.listdir(seg_path)
bunch.target_name.extend(catelist)
for mydir in catelist:
    class_path = seg_path+mydir+"/"
    file_list = os.listdir(class_path)
    for file_path in file_list:
        fullname = class_path + file_path
        bunch.label.append(mydir)
        bunch.filenames.append(fullname)
        bunch.contents.append(readfile(fullname).strip())
file_obj = open(wordbag_path,"wb")
pickle.dump(bunch,file_obj)   #以該格式(bunch)狀態(pickle)儲存於硬碟中
file_obj.close()
print("建置文字物件結束")

建置文字物件結束


## ```中文文字分類步驟三 : 向量空間模型```

- 向量空間模型為文字分類的結構化方法
- 向量空間模型將文字表示為一個向量，其特徵為文字中出現的詞
- 向量空間模型
    - 由於文字向量是一個高維空間，為了降低維度必須先過濾掉停用詞

## ```中文文字分類步驟三 : 向量空間模型(程式範例)```

- 讀取停用詞檔案

In [7]:
stopword_path = "train_word_bag\hlt_stop_words.txt"
stpwrdlst = readfile(stopword_path).splitlines()

## ```中文文字分類步驟四 : TF-IDF加權策略```

- ```詞頻(TF)```
    - 指某一指定的詞在文中的出現次數(是一種歸一化的表現)
    $$n_{i,j} \over \sum\limits_{k=1}n_{k,j}$$
    - 其中$n_{i,j}$表示在文中的出現次數，$n_{k,j}$的加總為該文件中所有詞彙的總和
- ```逆向檔案頻率(IDF)```
    - 意義:當某一詞彙在該文件中頻繁出現，但在其他文件中較少出現則表示該詞彙具有較高識別能力。應給予較高權重!反之應給予較少權重
    $$log{|D| \over |j:t_i \in d_j|}$$
    - 其中分子D為所有文件個數，分母為該詞在所有文件中的出現次數(一個詞在某文件中即便出現很多次也算一次)

## ```中文文字分類步驟四 : TF-IDF加權策略(範例)```

- 有三個文件
    - ```文件一:My dog ate my homework```
    - ```文件二:My cat ate the sandwich```
    - ```文件三:A dolphin ate the homework```
- 三個文件產生的詞頻
    - ```a(1次), ate(3次), cat(1次), dolphin(1次), dog(1次), homework(2次), my(3次), sandwich(1次), the(2次)```
- 經詞頻轉換後的三個文件
    - 文件一 : ```0, 1, 0, 0, 1, 1, 1, 0, 0```
    - 文件二 : ```0, 1, 1, 0, 0, 0, 1, 1, 1```
    - 文件三 : ```1, 1, 0, 1, 0, 1, 0, 0, 1```
- 上述的詞頻的轉換方式會忽略掉詞彙在文件中出現的次數，因此正確轉換如下
    - 文件一 : ```0, 1, 0, 0, 1, 1, 2, 0, 0 (my在文件中出現2次)```
    - 文件二 : ```0, 1, 1, 0, 0, 0, 1, 1, 1```
    - 文件三 : ```1, 1, 0, 1, 0, 1, 0, 0, 1```
- 為了避免句子長度的不一致，因此使用歸一化 ( 這就是TF )
    - 文件一 : ```0, 1/5, 0, 0, 1/5, 1/5, 2/5, 0, 0 (my在文件中出現2次)```
    - 文件二 : ```0, 1/5, 1/5, 0, 0, 0, 1/5, 1/5, 1/5```
    - 文件三 : ```1/5, 1/5, 0, 1/5, 0, 1/5, 0, 0, 1/5```

## ```中文文字分類步驟四 : TF-IDF加權策略(範例)```

- 詞頻的歸一化(詞頻是所有文件中詞彙出現的次數，因此歸一化分母應擺文件數)
    - ```a(1次/3份文件), ate(3次/3份文件), cat(1次/3份文件), dolphin(1次/3份文件), dog(1次/3份文件), homework(2次/3份文件), my(3次/3份文件), sandwich(1次), the(2次/3份文件)```
- 逆向檔案頻率(IDF)
    - ```a log(3/1), ate log(3/3), cat log(3/1), dolphin log(3/1), dog log(3/1), homework log(3/2), my log(3份文件/2份文件裡有出現), sandwich log(3/1), the log(3/2)```
- 最後再計算 TF與IDF 的乘積。TF-IDF偏好過濾掉常見的詞語，保留重要的詞語。

## ```中文文字分類步驟四 : TF-IDF加權策略(程式範例)```

- 匯入所需Scikit-Learn套件

In [26]:
import sys
import os
from sklearn.datasets.base import Bunch
import pickle
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

## ```中文文字分類步驟四 : TF-IDF加權策略(程式範例)```

- 讀取和寫入Bunch物件的函數

In [27]:
# 讀取Bunch狀態下的pickle物件

def readbunchobj(path):
    file_obj = open(path, "rb")
    bunch = pickle.load(file_obj)
    file_obj.close()
    return bunch

# 寫入Bunch狀態下的pickle物件

def writebunchobj(path,bunchobj):
    file_obj = open(path, "wb")
    pickle.dump(bunchobj,file_obj)
    file_obj.close()

## ```中文文字分類步驟四 : TF-IDF加權策略(程式範例)```

- 從訓練集產生TF-IDF向量詞袋

In [10]:
# 匯入分詞後的詞向量Bunch物件

path = "train_word_bag/train_set.dat"
bunch = readbunchobj(path)
# 建置TF-IDF詞向量空間物件

tfidfspace = Bunch(target_name=bunch.target_name, label=bunch.label, filenames=bunch.filenames, tdm=[], vocabulary={})

# 使用TfidfVectorizer初始化向量空間模型

vectorizer = TfidfVectorizer(stop_words=stpwrdlst, sublinear_tf=True, max_df=0.5)
transformer=TfidfTransformer()     #計算每個詞語的TF-IDF

# 文字轉為詞頻矩陣，單獨儲存字典檔案

tfidfspace.tdm = vectorizer.fit_transform(bunch.contents)
tfidfspace.vocabulary = vectorizer.vocabulary_

## ```中文文字分類步驟四 : TF-IDF加權策略(程式範例)```

- 持久化TF-IDF向量詞袋

In [11]:
# 建立詞袋的持久化

space_path = "train_word_bag/tfdifspace.dat"
writebunchobj(space_path,tfidfspace)

## `中文文字分類步驟五 : 分類器（1）`

- 中文的分類方法有以下幾種
    - kNN最近鄰演算法 : 原理簡單，分類精度尚可，但速度很慢。
    - 單純貝氏分類 : 分類短文字效果好，精度高。
    - 支援向量機 : 支援線性不可分的情況，精度尚可。
- 這次的實驗將使用Scikit-Learn中的單純貝氏演算法進行分類

## `中文文字分類步驟五 : 分類器（2）`

- 在使用分類器建模之前，必須先對測試資料進行整理。整理的方式與訓練資料相同
- 首先先將已經分詞好的測試資料使用Bunch結構化並以Pickle儲存於本機

In [28]:
from sklearn.datasets.base import Bunch
import pickle

bunch = Bunch(target_name=[],label=[],filenames=[],contents=[]) #宣告物件格式

wordbag_path = "test_word_bag/test_set.dat"    #格式化之後儲存的路徑，以pickle的方式儲存
seg_path = "test_corpus_seg/"       #分詞後，要進行格式化的檔案路徑

catelist = os.listdir(seg_path)
bunch.target_name.extend(catelist)
for mydir in catelist:
    class_path = seg_path+mydir+"/"
    file_list = os.listdir(class_path)
    for file_path in file_list:
        fullname = class_path + file_path
        bunch.label.append(mydir)
        bunch.filenames.append(fullname)
        bunch.contents.append(readfile(fullname).strip())
file_obj = open(wordbag_path,"wb")
pickle.dump(bunch,file_obj)   #以該格式(bunch)狀態(pickle)儲存於硬碟中
file_obj.close()
print("建置文字物件結束")

建置文字物件結束


## `中文文字分類步驟五 : 分類器（3）`
- 將Bunch物件化的測試資料集對映到訓練資料集的詞袋，並建置TF-IDF向量空間

In [29]:
# 匯入分詞後的詞向量Bunch物件
path = "test_word_bag/test_set.dat"
bunch = readbunchobj(path)
#建置測試集TF-IDF向量空間
testspace = Bunch(target_name=bunch.target_name, label=bunch.label, filenames=bunch.filenames, tdm=[], vocabulary={})
#匯入訓練及詞袋
trainbunch = readbunchobj("train_word_bag/tfdifspace.dat")
#使用TfidfVectorizer初始化向量空間模型
vectorizer = TfidfVectorizer(stop_words=stpwrdlst, sublinear_tf=True, max_df=0.5, vocabulary=trainbunch.vocabulary)#使用訓練集詞袋
transformer=TfidfTransformer()
testspace.tdm = vectorizer.fit_transform(bunch.contents)
testspace.vocabulary = trainbunch.vocabulary
#建立詞袋持久化
space_path = "test_word_bag/testspace.dat"
writebunchobj(space_path,testspace)

## `中文文字分類步驟五 : 分類器（4）`

- 執行貝氏分類，對測試文字進行分類，並回傳分類精度

In [30]:
from sklearn.naive_bayes import MultinomialNB

# 匯入資料集向量空間
trainpath = "train_word_bag/tfdifspace.dat"
train_set = readbunchobj(trainpath)
# 匯入測試集向量空間
testpath = "test_word_bag/testspace.dat"
test_set = readbunchobj(testpath)
# 應用單純貝氏演算法(alpha:0.001，alpha越小反覆運算次數越多，精度越高)
clf = MultinomialNB(alpha = 0.001).fit(train_set.tdm, train_set.label)

## `中文文字分類步驟五 : 分類器（5）`
- 預測分類結果

In [38]:
#預測分類結果
predicted = clf.predict(test_set.tdm)
total = len(predicted)
rate=0
for flabel,file_name,expct_cate in zip(test_set.label,test_set.filenames,predicted):
    if flabel != expct_cate:
        rate+=1
        print(file_name,":實際類別:",flabel,"-->預測類別",expct_cate)
print("error rate:",float(rate)*100/float(total),"%")


test_corpus_seg/art/3143.txt :實際類別: art -->預測類別 education
error rate: 0.9900990099009901 %


## `中文文字分類步驟六 : 評估分類結果（1）`

- 分類模型的評估指標有以下三種
    - 召回率`(Recall)` : 所有正類的結果中，預測為正類的比率。即${TP} \over {TP+FN}$
    - 準確率`(Precision)`:所有預測為正類的結果中，確實是正類的比率${TP} \over {TP+FP}$。
    - `F-Measure(F-Score)`:分類評估器的標準價值。即${2\times P \times R} \over {P+R}$，其中`P是Precision、R是Recall`
![混淆矩陣](https://mirdex.github.io/Machine_Learning/混淆矩陣.JPG)

## `中文文字分類步驟六 : 評估分類結果（2）`
- 該文字分類的評估值

In [79]:
import numpy as np

from sklearn import metrics

def metrics_result(actual,predict):
    print ("Precision：%.3f" % metrics.precision_score(actual,predict,average='weighted'))
    print("Recall： %.3f" % metrics.recall_score(actual,predict,average='weighted'))
    print("F-Measure: %.3f" % metrics.f1_score(actual,predict,average='weighted'))

metrics_result(test_set.label,predicted)

Precision：0.991
Recall： 0.990
F-Measure: 0.990
